In [2]:
import os
from langchain_community.document_loaders import PyPDFDirectoryLoader
DOCS_DIR = './ustc-yjs-data'
loader = PyPDFDirectoryLoader(DOCS_DIR)
pages = loader.load_and_split()
pdf_list = os.listdir(DOCS_DIR)

In [3]:
pages

[Document(metadata={'source': 'ustc-yjs-data/Rule_ustc_yjs.pdf', 'page': 0}, page_content='- 1 - \n  \n \n \n \n \n校研字〔 2023〕76号 \n \n \n \n \n \n \n各院、系、重点科研机构、直属单位、附属医院，机关各部门：  \n为规范研究生学籍管理，维护学校正常的教育教学秩序，保\n障广大研究生合法权益，全面贯彻国家教育方针，培养德智体美\n劳全面发展的社会主义建设者和接班人，依据《普通高等学校学\n生管理规定》 （教育部第 41号令） 以及 《中国科学技术大学章程》 ，\n结合学校研究生教育具体情况，学校对《中国科学技术大学研究\n生学籍管理实施细则》进行了修订，经 2023年3月15日校长工\n作会议审议通过，现予以印发施行。'),
 Document(metadata={'source': 'ustc-yjs-data/Rule_ustc_yjs.pdf', 'page': 1}, page_content='- 2 - 特此通知。  \n \n \n \n \n中国科学 技术大学  \n2023年5月28日'),
 Document(metadata={'source': 'ustc-yjs-data/Rule_ustc_yjs.pdf', 'page': 2}, page_content='- 3 - \n \n \n第一章  总  则 \n第一条  为规范我校研究生学籍管理，维护学校正常的教育\n教学秩序，保障广大研究生合法权益，全面贯彻国家教育方针，\n培养德智体美劳全面发展的社会主义建设者和接班人，依据《普\n通高等学校学生管理规定》 （教育部第 41号令）以及《中国科学\n技术大学章程》 ，结合学校研究生教育具体情况 ,制定本规定。  \n第二章  入学与注册  \n第二条  按国家招生规定录取的研 究生新生， 持录取通知\n书，按学校 要求办理入学手续。因故不能按期入学的，应当向学\n校请假。请假时间不得超过 2周。未请假或请假逾期者，除因不\n可抗力等正当事由以外，视为放弃入学资格。  \n第三条  学校在报到时对新生入学资格进行初步审查，审查\n合格的办理入学手续， 予以注册学籍； 审查发现新生的录取通知、\n

In [4]:
from tqdm import tqdm
pdf_text = { pdf_page.metadata['source'][-17:]:'' for pdf_page  in pages }
for pdf in tqdm(pdf_list):
    for pdf_page in pages:
        if pdf in pdf_page.metadata['source']:
            pdf_text[pdf] += pdf_page.page_content
        else:
            continue
print('key:pdf value:text')

100%|██████████| 1/1 [00:00<00:00, 8774.69it/s]

key:pdf value:text


In [5]:
with open('rule_ustc_yjs.txt','w',encoding = 'utf-8') as file:
    pdf_all = ''.join(list(pdf_text.values())).encode('utf-8', 'replace').decode('utf-8')
    file.write( pdf_all)  

In [6]:
import re

def split_entries(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # 清理页码（如"- 6 -"）
    content = re.sub(r'-\s*\d+\s*-', '', content)  # 关键步骤

    # 在读取后先清除章节标题
    content = re.sub(r'^第[一二三四五六七八九十]+章\s+.+\n', '', content, flags=re.MULTILINE)
    content = content.replace('\r\n', '\n').replace('\r', '\n')  # 统一换行符
    
    # 改进正则：匹配"第x条"且后跟多个空格（标题特征）
    pattern = r'第[一二三四五六七八九十零百千万]+条\s+'
    matches = list(re.finditer(pattern, content, flags=re.MULTILINE))
    
    entries = []
    for i in range(len(matches)):
        start = matches[i].start()
        end = matches[i+1].start() if i < len(matches)-1 else len(content)
        entries.append(content[start:end].strip())  # 去除首尾空白
    
    return entries

# 使用示例
entries = split_entries('rule_ustc_yjs.txt')

In [7]:
entries

['第一条  为规范我校研究生学籍管理，维护学校正常的教育\n教学秩序，保障广大研究生合法权益，全面贯彻国家教育方针，\n培养德智体美劳全面发展的社会主义建设者和接班人，依据《普\n通高等学校学生管理规定》 （教育部第 41号令）以及《中国科学\n技术大学章程》 ，结合学校研究生教育具体情况 ,制定本规定。',
 '第二条  按国家招生规定录取的研 究生新生， 持录取通知\n书，按学校 要求办理入学手续。因故不能按期入学的，应当向学\n校请假。请假时间不得超过 2周。未请假或请假逾期者，除因不\n可抗力等正当事由以外，视为放弃入学资格。',
 '第三条  学校在报到时对新生入学资格进行初步审查，审查\n合格的办理入学手续， 予以注册学籍； 审查发现新生的录取通知、\n考生信息等证明材料，与本人实际情况不符，或者有其他违反国\n家招生考试规定情形的，取消入学资格。',
 '第四条  新生可以申请保留入学资格。保留入学资格期间不\n具有学籍，不享受在校生待遇。除参 军入伍外，申请保留入学资\n格每次时间不超过 1年，累计时间不超过 2年。 新生保留入学资格应提交《研究生保留入学资格申请表》 ，\n附事由证明材料，经导师、培养单位分管领导书面同意，报研究\n生院审批办理保留手续。因病保留入学资格者，应提交经学校指\n定的二级甲等以上医院（下同）诊断证明。  \n新生保留入学资格期满前应向学校申请入学，提交《研究生\n恢复入学资格申请表》 ，附事由证明材料，经导师、培养单位分\n管领导书面同意，报研究生院审批。因病保留入学资格者，应提\n交学校指定医院诊断证明。经学校审查合格后，办理入学手续，\n予以注册学籍。 以学年 为时间单位保留的， 与下年新生同时入学。\n学校审查不合格的，取消入学资格；新生逾期不办理入学手续且\n未有因不可抗力延迟等正当理由的，视为放弃入学资格。',
 '第五条  新生入学后，学校在 3个月内按照国家招生规定进\n行复查。复查内容主要包括以下方面：  \n（一）录取手续及程序等是否合乎国家招生规定；  \n（二）所获得的录取资格是否真实、合乎相关规定；  \n（三）本人及身份证明与录取通知、考生档案等是否一致；  \n（四）身心健康状况是否符合报考专业或者专业类别体检要\n求，能否保证在校正常学习、生活；  \n（五）艺术、体育等特殊类型录取学生的

In [ ]:
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import torch

DEVICE = "cuda:6"

class FaissRetriever(object):
    def __init__(self, data):
        # 使用固定的 bge-large-zh-v1.5 模型，不允许微调
        self.embeddings = HuggingFaceEmbeddings(
            model_name="/data/DeHors_yh_0329/code/RAG/DS2024-BDCI-RAG/src/BAAI/bge-large-zh-v1.5",  # 指定模型为 bge-large-zh-v1.5
            model_kwargs={"device": DEVICE}       # 使用 GPU 进行推理
        )
        docs = []
        for idx, line in enumerate(data):
            line = line.strip("\n").strip()
            words = line.split("\t")
            docs.append(Document(page_content=words[0], metadata={"id": idx}))

        # 使用 FAISS 进行向量存储
        self.vector_store = FAISS.from_documents(docs, self.embeddings)

    def GetTopK(self, query, k):
        # 基于查询找到最相似的前 k 个文档
        context = self.vector_store.similarity_search_with_score(query, k=k)
        return context

    def GetvectorStore(self):
        return self.vector_store


In [11]:
faissretriever = FaissRetriever(entries)
vector_store = faissretriever.vector_store

In [12]:
from langchain.retrievers import BM25Retriever
from langchain.schema import Document

import jieba

class BM25(object):

    def __init__(self, documents):

        docs = []
        full_docs = []
        for idx, line in enumerate(documents):
            line = line.strip("\n").strip()
            if(len(line)<5):
                continue
            tokens = " ".join(jieba.cut_for_search(line))
            # docs.append(Document(page_content=tokens, metadata={"id": idx, "cate":words[1],"pageid":words[2]}))
            docs.append(Document(page_content=tokens, metadata={"id": idx}))
            # full_docs.append(Document(page_content=words[0], metadata={"id": idx, "cate":words[1], "pageid":words[2]}))
            words = line.split("\t")
            full_docs.append(Document(page_content=words[0], metadata={"id": idx}))
        self.documents = docs
        self.full_documents = full_docs
        self.retriever = self._init_bm25()

    # 初始化BM25的知识库
    def _init_bm25(self):
        return BM25Retriever.from_documents(self.documents)

    # 获得得分在topk的文档和分数
    def GetBM25TopK(self, query, topk):
        self.retriever.k = topk
        query = " ".join(jieba.cut_for_search(query))
        ans_docs = self.retriever.get_relevant_documents(query)
        ans = []
        for line in ans_docs:
            ans.append(self.full_documents[line.metadata["id"]])
        return ans


In [13]:
bm25 = BM25(entries)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.706 seconds.
Prefix dict has been built successfully.


In [16]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import os

def torch_gc():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()

class reRankLLM(object):
    def __init__(self, model_path="/data/DeHors_yh_0329/code/RAG/DS2024-BDCI-RAG/src/BAAI/bge-reranker-large", max_length=512):
        # 使用指定的模型路径 BAAI/bge-reranker-large
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_path)
        self.model.eval()
        self.model.half()
        self.model.to(DEVICE)  # 将模型移动到指定的设备
        self.max_length = max_length

    def predict(self, query, docs):
        # 创建 query 和文档内容对
        pairs = [(query, doc.page_content) for doc in docs]
        # Tokenize 输入内容，并将其移动到 GPU
        inputs = self.tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=self.max_length).to(DEVICE)
        
        with torch.no_grad():
            # 使用模型进行推理并计算得分
            scores = self.model(**inputs).logits
        
        # 将得分从 GPU 移动到 CPU
        scores = scores.detach().cpu().clone().numpy()
        # 根据得分对文档排序
        scored_docs = sorted(zip(scores, docs), reverse=True, key=lambda x: x[0])
        response = [doc for score, doc in scored_docs]

        # 打印查询和对应的前三个文档及其分数
        print(f"Query: {query}")
        for i, (score, doc) in enumerate(scored_docs[:3]):
            print(f"Rank {i + 1}: Score: {score[0]}, Document: {doc.page_content[:50]}...")  # 仅打印文档的前0个字符
        
        # torch_gc()  # 清理缓存
        return response


In [17]:
rerank = reRankLLM()

In [18]:
def reRank(rerank, top_k, query, bm25_ans, faiss_ans):
    items = []
    max_length = 4000
    for doc, score in faiss_ans:
        items.append(doc)
    items.extend(bm25_ans)
    rerank_ans = rerank.predict(query, items)
    rerank_ans = rerank_ans[:top_k]
    emb_ans = ""
    for doc in rerank_ans:
        if(len(emb_ans + doc.page_content) > max_length):
            break
        emb_ans = emb_ans + doc.page_content
    return emb_ans

In [20]:
import pandas as pd
QA_df = pd.read_csv('./ques_ans.csv', sep=',')

In [61]:
import pandas as pd

# 假设 submit_df 已经在之前的代码中定义并包含问题
# 提取问题列表
queries = QA_df['问'].tolist()
answers = QA_df['答'].tolist()

# 初始化一个字典来存储每个列的数据
data = {
    'query': [],  # 添加查询字段
    'answer': [],  # 添加答案字段
    'faiss_context': [],  # 保存 faiss 检索结果
    'bm25_context': [],  # 保存 bm25 检索结果
    'emb_bm25_merge_inputs': []  # 合并后的输入
}

max_length = 4000

# 遍历每个查询
for idx, query in enumerate(queries):
    # faiss 检索
    faiss_context = faissretriever.GetTopK(query, 15)
    faiss_min_score = 0.0
    
    if len(faiss_context) > 0:
        faiss_min_score = faiss_context[0][1]
    
    cnt = 0
    emb_ans = ""
    
    for doc, score in faiss_context:
        cnt += 1
        if len(emb_ans + doc.page_content) > max_length:
            break
        emb_ans += doc.page_content
        if cnt > 6:
            break

    # bm25 检索
    bm25_context = bm25.GetBM25TopK(query, 15)
    bm25_ans = ""
    cnt = 0
    
    for doc in bm25_context:
        cnt += 1
        if len(bm25_ans + doc.page_content) > max_length:
            break
        bm25_ans += doc.page_content
        if cnt > 6:
            break

    # 合并 faiss 和 bm25 的输入
    rerank_ans = reRank(rerank, 1, query, bm25_context, faiss_context)
    
    # 将结果添加到数据字典中
    data['query'].append(query)  # 保存查询
    data['answer'].append(answers[idx])  # 保存答案
    data['faiss_context'].append(emb_ans)  # 保存 faiss 检索结果
    data['bm25_context'].append(bm25_ans)  # 保存 bm25 检索结果
    data['emb_bm25_merge_inputs'].append(rerank_ans)  # 保存合并结果

# 将字典转换为 DataFrame
batch_input_df = pd.DataFrame(data)


Query: 研究生新生因故不能按期入学怎么办？
Rank 1: Score: 6.49609375, Document: 第二条  按国家招生规定录取的研 究生新生， 持录取通知
书，按学校 要求办理入学手续。因故不能按期...
Rank 2: Score: 6.49609375, Document: 第二条  按国家招生规定录取的研 究生新生， 持录取通知
书，按学校 要求办理入学手续。因故不能按期...
Rank 3: Score: 4.49609375, Document: 第二十一条   研究生应当按时参加教育教学计划规定的活
动。不能按时参加的，应当事先请假并获得批准。...
Query: 除参军入伍外，申请保留入学资格的时间有什么要求？
Rank 1: Score: 4.98828125, Document: 第四条  新生可以申请保留入学资格。保留入学资格期间不
具有学籍，不享受在校生待遇。除参 军入伍外，...
Rank 2: Score: 4.98828125, Document: 第四条  新生可以申请保留入学资格。保留入学资格期间不
具有学籍，不享受在校生待遇。除参 军入伍外，...
Rank 3: Score: 2.65234375, Document: 第十三条   我校硕士研究生基本学习年限为 2-3年（年限计 
 算截至每年 8月31日，下同） ，...
Query: 研究生在每学期开学时办理注册手续的方式有哪些？ 
Rank 1: Score: 5.8984375, Document: 第六条  研究生在学期间，每学期开学时应当按学校规定办
理注册手续，包括学期电子注册、研究生证注册。...
Rank 2: Score: 5.8984375, Document: 第六条  研究生在学期间，每学期开学时应当按学校规定办
理注册手续，包括学期电子注册、研究生证注册。...
Rank 3: Score: 3.88671875, Document: 第九条  处于保留入学资格、休学或保留学籍期间的研究
生，无须办理学期注册。学期中非注册时间段内，暂...
Query: 哪些情况属于研究生的正常注册？ 
Rank 1: Score: 5.75390625, Document: 第七条  学期电子注册是指学校通过研究生信息平台进行
的电子注册管

In [63]:
batch_input_df

,query,answer,faiss_context,bm25_context,emb_bm25_merge_inputs
0,研究生新生因故不能按期入学怎么办？,应当向学校请假，假期不得超过2周，未请假或请假逾期的除不可抗力原因外视为放弃入学资格。,第二条 按国家招生规定录取的研 究生新生， 持录取通知\n书，按学校 要求办理入学手续。因...,第二条 按国家招生规定录取的研 究生新生， 持录取通知\n书，按学校 要求办理入学手续。因...,第二条 按国家招生规定录取的研 究生新生， 持录取通知\n书，按学校 要求办理入学手续。因...
1,除参军入伍外，申请保留入学资格的时间有什么要求？,新生可以申请保留入学资格，每次时间不超过1年，累计不超过2年。,第四条 新生可以申请保留入学资格。保留入学资格期间不\n具有学籍，不享受在校生待遇。除参 ...,第十三条 我校硕士研究生基本学习年限为 2-3年（年限计 \n 算截至每年 8月31日，...,第四条 新生可以申请保留入学资格。保留入学资格期间不\n具有学籍，不享受在校生待遇。除参 ...
2,研究生在每学期开学时办理注册手续的方式有哪些？,研究生在学期间，每学期开学时应当按学校规定办理注册手续，包括学期电子注册、研究生证注册。,第六条 研究生在学期间，每学期开学时应当按学校规定办\n理注册手续，包括学期电子注册、研究...,第六条 研究生在学期间，每学期开学时应当按学校规定办\n理注册手续，包括学期电子注册、研究...,第六条 研究生在学期间，每学期开学时应当按学校规定办\n理注册手续，包括学期电子注册、研究...
3,哪些情况属于研究生的正常注册？,研究生以下情况均视为正常注册：（一）到校学习；（二）经学校批准出境学习或交流；（三）经导师批...,第七条 学期电子注册是指学校通过研究生信息平台进行\n的电子注册管理。由研究生通过研究生信...,第八条 除以上情况外，研究生不能如期注册，从开学日起\n请假2周以内的，由本人通过研究生信...,第七条 学期电子注册是指学校通过研究生信息平台进行\n的电子注册管理。由研究生通过研究生信...
4,研究生不能如期注册，从开学日起请假2 周以上或不满足其他注册必要条件的，应如何处理？,应当履行暂缓注册手续，提交《研究生暂缓注册申请表》，经导师或班主任、培养单位分管领导书面同意...,第十条 除因不可抗力等正当事由外，研究生超过注册 期限\n或请假期限 2周以上未注册，学校...,第八条 除以上情况外，研究生不能如期注册，从开学日起\n请假2周以内的，由本人通过研究生信...,第八条 除以上情况外，研究生不能如期注册，从开学日起\n请假2周以内的，由本人通过研究生信...
5,我校研究生的基本学习年限是多少？,我校硕士研究生基本学习年限为 2-3 年（年限计算截至每年 8 月 31 日，下同），最短学...,第十四条 研究生一般应在基本学习年限内完成学习任务。\n在校学习时间达到最短学习年限且不...,第十四条 研究生一般应在基本学习年限内完成学习任务。\n在校学习时间达到最短学习年限且不...,第十三条 我校硕士研究生基本学习年限为 2-3年（年限计 \n 算截至每年 8月31日，...
6,课程考核不合格的研究生可以怎样做？,可以在最长学习年限内再次选修该门课程（重修）， 并进行考核（重考）。重考成绩合格后，记入成绩...,第十五条 研究生应参加学校培养计划规定的课程和各种\n培养环节（以下统称课程）的考核，考...,第十五条 研究生应参加学校培养计划规定的课程和各种\n培养环节（以下统称课程）的考核，考...,第十五条 研究生应参加学校培养计划规定的课程和各种\n培养环节（以下统称课程）的考核，考...
7,有哪些情况下，研究生不得申请转学？,研究生有下列情形之一，不得转学：（一）入学未满一学期或者毕业前一年的；（二）由低学历层次转为...,第二十四条 研究生一般应当在被录取学校完成学业。因患\n病或者有特殊困难、特别需要，无法...,第二十四条 研究生一般应当在被录取学校完成学业。因患\n病或者有特殊困难、特别需要，无法...,第二十四条 研究生一般应当在被录取学校完成学业。因患\n病或者有特殊困难、特别需要，无法...
8,哪些情况下，研究生可以申请休学？,研究生有下列情况之一者，可以申请休学：（一）创业；（二）支教；（三）在职研究生单位工作需要；...,第二十八条 研究生有下列情况之一 者，可以申请休学： \n（一）创业； \n（二）支...,第二十三条 研究生应在录取时确定的培养单位、专业和导\n师指导下完成学业，有特殊情况的，...,第二十八条 研究生有下列情况之一 者，可以申请休学： \n（一）创业； \n（二）支...
9,研究生因健康原因不宜在校学习应如何处理？,经指定医院诊断确需休养的，应办理休学手续。,第二十九条 研究生有下列情况之一者，应当办理休学： \n （一）因健康原因不宜在校学习，...,第二十九条 研究生有下列情况之一者，应当办理休学： \n （一）因健康原因不宜在校学习，...,第二十九条 研究生有下列情况之一者，应当办理休学： \n （一）因健康原因不宜在校学习，...


In [1]:
from langchain_community.llms import SparkLLM
# 通过api调用大模型
llm = SparkLLM(
    spark_app_id="YOURID",
    spark_api_key="YOURKEY",
    spark_api_secret="YOURSECRET",
    spark_api_url="wss://spark-api.xf-yun.com/v1.1/chat",
    model="lite"
)

In [2]:
res = llm.invoke("What's your name?")
res
# print(response)

'My name is iFLYTEK Spark. How can I assist you today?'

In [11]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
augmented_prompt = """[角色]
你是一个严谨的问答专家，严格依据提供的知识库内容进行回答

[指令]
请按以下步骤处理：
1. 上下文核查：分析<上下文>与<问题>的关联性
2. 知识提取：仅使用提供的上下文信息，禁止外部知识
3. 回答组织：
   - 若上下文包含明确答案：可以直接使用原文，然后剩余内容根据相关性进行补充。
   - 若上下文不相关/不充分：明确告知无法回答
   - 始终在结尾使用：期待为您提供更多帮助

[输出格式要求]
回答必须包含：
- 首段：直接的问题回答（如原文内容），如果是列点回答，则要完整列出所有点（子项）
- 末段：剩余内容根据相关性进行补充

### 上下文 ###
{source_knowledge}

### 问题 ###
{query}
有用的回答:"""

prompt = PromptTemplate(template=augmented_prompt, input_variables=["source_knowledge" ,"query"])
llm_chain = LLMChain(prompt=prompt, llm=llm  , llm_kwargs = {"temperature":0, "max_length":1024})

In [65]:
# 假设 batch_input_df 已经在之前的代码中定义并包含 question 和 emb_bm25_merge_inputs
# 初始化一个空列表来存储 LLM 的回答
llm_answers = []

# 遍历每个查询和对应的源知识
for idx, row in batch_input_df.iterrows():
    query = row['query']
    source_knowledge = row['emb_bm25_merge_inputs']
    
    # 使用 SparkLLM 生成回答
    llm_answer = llm_chain.run( {"source_knowledge":source_knowledge ,"query" : query })
    
    # 将生成的回答添加到列表中
    llm_answers.append(llm_answer)

# 将生成的回答添加到 batch_input_df 中的新列 LLM_answer
batch_input_df['LLM_answer'] = llm_answers

In [66]:
selected_columns = batch_input_df[['query', 'answer', 'LLM_answer']]

In [68]:
selected_columns

,query,answer,LLM_answer
0,研究生新生因故不能按期入学怎么办？,应当向学校请假，假期不得超过2周，未请假或请假逾期的除不可抗力原因外视为放弃入学资格。,研究生新生因故不能按期入学，应当向学校请假。请假时间不得超过2周。未请假或请假逾期者，除因不...
1,除参军入伍外，申请保留入学资格的时间有什么要求？,新生可以申请保留入学资格，每次时间不超过1年，累计不超过2年。,根据提供的上下文信息，除参军入伍外，申请保留入学资格的时间要求是每次不超过1年，累计时间不超...
2,研究生在每学期开学时办理注册手续的方式有哪些？,研究生在学期间，每学期开学时应当按学校规定办理注册手续，包括学期电子注册、研究生证注册。,- 学期电子注册：在每学期开学时，研究生需要使用学校指定的电子系统进行学期电子注册。\n- ...
3,哪些情况属于研究生的正常注册？,研究生以下情况均视为正常注册：（一）到校学习；（二）经学校批准出境学习或交流；（三）经导师批...,研究生的正常注册情况包括： \n（一）到校学习； \n（二）经学校批准出境学习或交流； ...
4,研究生不能如期注册，从开学日起请假2 周以上或不满足其他注册必要条件的，应如何处理？,应当履行暂缓注册手续，提交《研究生暂缓注册申请表》，经导师或班主任、培养单位分管领导书面同意...,- 研究生不能如期注册，从开学日起请假2周以上或不满足其他注册必要条件的，应当履行暂缓注册手...
5,我校研究生的基本学习年限是多少？,我校硕士研究生基本学习年限为 2-3 年（年限计算截至每年 8 月 31 日，下同），最短学...,根据所提供的上下文，我校硕士研究生的基本学习年限为2-3年。博士研究生的基本学习年限为3-4...
6,课程考核不合格的研究生可以怎样做？,可以在最长学习年限内再次选修该门课程（重修）， 并进行考核（重考）。重考成绩合格后，记入成绩...,课程考核不合格的研究生，在最长学习年限内，可以选择再次选修该课程（重修），并进行考核（重考）...
7,有哪些情况下，研究生不得申请转学？,研究生有下列情形之一，不得转学：（一）入学未满一学期或者毕业前一年的；（二）由低学历层次转为...,- 入学未满一学期或者毕业前一年的；\n- 由低学历层次转为高学历层次的；\n- 以定向就业...
8,哪些情况下，研究生可以申请休学？,研究生有下列情况之一者，可以申请休学：（一）创业；（二）支教；（三）在职研究生单位工作需要；...,研究生可以申请休学的情况包括：\n\n1. 创业；\n2. 支教；\n3. 在职研究生单位工...
9,研究生因健康原因不宜在校学习应如何处理？,经指定医院诊断确需休养的，应办理休学手续。,- 因健康原因不宜在校学习，经学校指定医院诊断确需休养的；


In [67]:
import datetime

# 可以按日期-时间保存，例如 "2024-10-01 21:28" 记为 "1001_2128"
today = datetime.datetime.now().strftime("%m%d_%H%M")
savepath = f'./rag_output_{today}.csv'
selected_columns.to_csv(savepath, sep=',', index=False)
print(f"Saved path: {savepath}")

Saved path: ./rag_output_0322_1535.csv


# 评测指标
在一般论文中，采用的一般是英文数据集，以多跳问答数据集为主，如HotpotQA。其中一般采用EM，F1，Accuracy等指标进行评测。
下面是相关代码实现：

```python   
def normalize_answer(s):
    """
    Normalizes the answer string.

    This function standardizes the answer string through a series of steps including removing articles,
    fixing whitespace, removing punctuation, and converting text to lowercase. This ensures consistency
    and fairness when comparing answers.

    Parameters:
    s (str): The answer string to be standardized.

    Returns:
    str: The standardized answer string.
    """

    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return str(text).lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    """
    Calculates the F1 score between the predicted answer and the ground truth.

    The F1 score is the harmonic mean of precision and recall, used to evaluate the model's performance in question answering tasks.

    Parameters:
    prediction (str): The predicted answer from the model.
    ground_truth (str): The actual ground truth answer.

    Returns:
    tuple: A tuple containing the F1 score, precision, and recall.
    """

    normalized_prediction = normalize_answer(prediction)
    normalized_ground_truth = normalize_answer(ground_truth)

    ZERO_METRIC = (0, 0, 0)

    if (
        normalized_prediction in ["yes", "no", "noanswer"]
        and normalized_prediction != normalized_ground_truth
    ):
        return ZERO_METRIC

    if (
        normalized_ground_truth in ["yes", "no", "noanswer"]
        and normalized_prediction != normalized_ground_truth
    ):
        return ZERO_METRIC

    prediction_tokens = normalized_prediction.split()
    ground_truth_tokens = normalized_ground_truth.split()

    # Calculate the number of matching words between the predicted and ground truth answers
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())

    if num_same == 0:
        return ZERO_METRIC

    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)

    return f1, precision, recall


def exact_match_score(prediction, ground_truth):
    """
    Calculates the exact match score between a predicted answer and the ground truth answer.

    This function normalizes both the predicted answer and the ground truth answer before comparing them.
    Normalization is performed to ensure that non-essential differences such as spaces and case are ignored.

    Parameters:
    prediction (str): The predicted answer string.
    ground_truth (str): The ground truth answer string.

    Returns:
    int: 1 if the predicted answer exactly matches the ground truth answer, otherwise 0.
    """

    return 1 if normalize_answer(prediction) == normalize_answer(ground_truth) else 0
```

其中首先通过标准化函数：
- 去除冠词：使用正则表达式去除 "a"、"an" 和 "the"。
- 修复空格：将多余的空格替换为单个空格。
- 去除标点：移除所有标点符号。
- 转换为小写：将文本转换为小写字母

然后EM：如果它们完全相等，返回 1；否则返回 0

F1 分数、精确度和召回率：
- 检查特殊情况（如 "yes"、"no" 和 "noanswer"），如果预测和真实答案不匹配，则返回零分。
- 将标准化后的答案分割为单词，计算预测和真实答案之间的共同单词数量。
- 计算精确度和召回率，并基于这两个值计算 F1 分数。

In [1]:
# 导入csv文件
import pandas as pd

data = pd.read_csv('./rag_output_0322_1535.csv')


In [6]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
augmented_prompt = """[角色]
你是一个严谨的问答专家，请判断LLM经过RAG生成的答案是否符合正确答案

[指令]
请按以下步骤处理：
1. 理解问题和答案
2. 判断LLM经过RAG生成的答案和答案是否符合
3. 判断原则：
   - 若LLM经过RAG生成的答案的语义与答案的语义一致，输出Yes。
   - 若LLM经过RAG生成的答案的语义包含答案的语义，输出Yes。
   - 若LLM经过RAG生成的答案的语义与答案的语义不一致，输出No。

[输出格式要求]
Yes or No

### 问题 ###
{query}

### 答案 ###
{answer}

### LLM经过RAG生成的答案 ###
{LLM_answer}
输出:"""

prompt = PromptTemplate(template=augmented_prompt, input_variables=["query" ,"answer", "LLM_answer"])
llm_chain = LLMChain(prompt=prompt, llm=llm  , llm_kwargs = {"temperature":0, "max_length":1024})

In [7]:
llm_judges = []

# 遍历每个查询和对应的源知识
for idx, row in data.iterrows():
    query = row['query']
    answer = row['answer']
    LLM_answer = row['LLM_answer']
    
    # 使用 SparkLLM 生成回答
    llm_judge = llm_chain.run( {"query":query ,"answer" : answer, "LLM_answer": LLM_answer })
    
    # 将生成的回答添加到列表中
    llm_judges.append(llm_judge)

# 将生成的回答添加到 batch_input_df 中的新列 LLM_answer
data['llm_judge'] = llm_judges

/tmp/ipykernel_1713513/2666286646.py:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm_judge = llm_chain.run( {"query":query ,"answer" : answer, "LLM_answer": LLM_answer })


In [8]:
data

,query,answer,LLM_answer,llm_judge
0,研究生新生因故不能按期入学怎么办？,应当向学校请假，假期不得超过2周，未请假或请假逾期的除不可抗力原因外视为放弃入学资格。,研究生新生因故不能按期入学，应当向学校请假。请假时间不得超过2周。未请假或请假逾期者，除因不...,Yes
1,除参军入伍外，申请保留入学资格的时间有什么要求？,新生可以申请保留入学资格，每次时间不超过1年，累计不超过2年。,根据提供的上下文信息，除参军入伍外，申请保留入学资格的时间要求是每次不超过1年，累计时间不超...,Yes
2,研究生在每学期开学时办理注册手续的方式有哪些？,研究生在学期间，每学期开学时应当按学校规定办理注册手续，包括学期电子注册、研究生证注册。,- 学期电子注册：在每学期开学时，研究生需要使用学校指定的电子系统进行学期电子注册。\n- ...,Yes
3,哪些情况属于研究生的正常注册？,研究生以下情况均视为正常注册：（一）到校学习；（二）经学校批准出境学习或交流；（三）经导师批...,研究生的正常注册情况包括： \n（一）到校学习； \n（二）经学校批准出境学习或交流； ...,Yes
4,研究生不能如期注册，从开学日起请假2 周以上或不满足其他注册必要条件的，应如何处理？,应当履行暂缓注册手续，提交《研究生暂缓注册申请表》，经导师或班主任、培养单位分管领导书面同意...,- 研究生不能如期注册，从开学日起请假2周以上或不满足其他注册必要条件的，应当履行暂缓注册手...,Yes
5,我校研究生的基本学习年限是多少？,我校硕士研究生基本学习年限为 2-3 年（年限计算截至每年 8 月 31 日，下同），最短学...,根据所提供的上下文，我校硕士研究生的基本学习年限为2-3年。博士研究生的基本学习年限为3-4...,Yes
6,课程考核不合格的研究生可以怎样做？,可以在最长学习年限内再次选修该门课程（重修）， 并进行考核（重考）。重考成绩合格后，记入成绩...,课程考核不合格的研究生，在最长学习年限内，可以选择再次选修该课程（重修），并进行考核（重考）...,Yes
7,有哪些情况下，研究生不得申请转学？,研究生有下列情形之一，不得转学：（一）入学未满一学期或者毕业前一年的；（二）由低学历层次转为...,- 入学未满一学期或者毕业前一年的；\n- 由低学历层次转为高学历层次的；\n- 以定向就业...,Yes
8,哪些情况下，研究生可以申请休学？,研究生有下列情况之一者，可以申请休学：（一）创业；（二）支教；（三）在职研究生单位工作需要；...,研究生可以申请休学的情况包括：\n\n1. 创业；\n2. 支教；\n3. 在职研究生单位工...,Yes
9,研究生因健康原因不宜在校学习应如何处理？,经指定医院诊断确需休养的，应办理休学手续。,- 因健康原因不宜在校学习，经学校指定医院诊断确需休养的；,Yes


In [18]:
import re
import jieba
import string
from collections import Counter

def normalize_answer_chinese(s):
    """
    中文标准化处理：去标点、修复空格、全角转半角、小写化（可选）
    """
    def remove_punc(text):
         # 中英文标点符号（可根据需求扩展）
        chinese_punctuation = '，。！？；：“”‘’（）【】《》…—·'
        all_punctuation = set(string.punctuation + chinese_punctuation)
        return ''.join(ch for ch in text if ch not in all_punctuation)

    def full_to_half(text):
        # 全角转半角（处理数字和英文字符）
        return text.translate(str.maketrans('１２３４５６７８９０ＡＢＣＤＥＦＧ',
                                          '1234567890ABCDEFH'))

    def white_space_fix(text):
        return " ".join(text.split())

    processed = full_to_half(s)          # 全角转半角
    processed = remove_punc(processed)   # 去标点
    processed = white_space_fix(processed)  # 修复空格
    return processed

def f1_score_chinese(prediction, ground_truth):
    """
    中文F1计算：基于分词后的词粒度匹配
    """
    normalized_pred = normalize_answer_chinese(prediction)
    normalized_gt = normalize_answer_chinese(ground_truth)

    # 处理特殊词（如“是/否/无答案”）
    SPECIAL_TOKENS = ["是", "否", "无答案"]
    if (normalized_pred in SPECIAL_TOKENS or normalized_gt in SPECIAL_TOKENS) \
        and normalized_pred != normalized_gt:
        return (0, 0, 0)

    # 中文分词
    pred_tokens = list(jieba.cut(normalized_pred, cut_all=False))
    gt_tokens = list(jieba.cut(normalized_gt, cut_all=False))

    common = Counter(pred_tokens) & Counter(gt_tokens)
    num_same = sum(common.values())

    if num_same == 0:
        return (0, 0, 0)

    precision = num_same / len(pred_tokens)
    recall = num_same / len(gt_tokens)
    f1 = (2 * precision * recall) / (precision + recall + 1e-10)  # 避免除零

    return (f1, precision, recall)

def exact_match_chinese(prediction, ground_truth):
    """
    中文精确匹配：标准化后完全一致返回1，否则0
    """
    return 1 if normalize_answer_chinese(prediction) == normalize_answer_chinese(ground_truth) else 0

In [ ]:
# def lenient_accuracy(examples, recall_threshold=0.8):
#     """
#     基于Recall的宽松准确率，研究生学籍管理问答系统不像医疗法律需要严格的精确度，提供的信息尽可能多包含更加适合的答案，因此使用Recall作为准确率的指标。
#     """
#     total = len(examples)
#     correct = 0
#     for _, row in examples.iterrows():
#         _, _, recall = f1_score_chinese(row['LLM_answer'], row['answer'])
#         if recall >= recall_threshold:
#             correct += 1
#     return correct / total

In [ ]:
total_metrics = {"em": 0.0, "f1": 0.0, "precision": 0.0, "recall": 0.0}

for idx, row in data.iterrows():
    answer = row['answer']
    llm_answer = row['LLM_answer']

    em = float(exact_match_chinese(llm_answer, answer))
    f1, precision, recall = f1_score_chinese(llm_answer, answer)

    total_metrics["em"] += em
    total_metrics["f1"] += f1
    total_metrics["precision"] += precision
    total_metrics["recall"] += recall
    print(f"EM: {em}, F1: {f1}, Precision: {precision}, Recall: {recall}")

total_metrics = {k: v / len(data) for k, v in total_metrics.items()}
print(total_metrics)

EM: 0.0, F1: 0.6666666666183814, Precision: 0.5625, Recall: 0.8181818181818182
EM: 0.0, F1: 0.41025641022232084, Precision: 0.26229508196721313, Recall: 0.9411764705882353
EM: 0.0, F1: 0.3999999999555556, Precision: 0.3, Recall: 0.6
EM: 0.0, F1: 0.8817204300576713, Precision: 0.8367346938775511, Recall: 0.9318181818181818
EM: 0.0, F1: 0.5641025640599605, Precision: 0.4074074074074074, Recall: 0.9166666666666666
EM: 0.0, F1: 0.3783783783397776, Precision: 0.7241379310344828, Recall: 0.25609756097560976
EM: 0.0, F1: 0.721311475361462, Precision: 0.6111111111111112, Recall: 0.88
EM: 0.0, F1: 0.8380952380454603, Precision: 0.8979591836734694, Recall: 0.7857142857142857
EM: 0.0, F1: 0.5671641790544887, Precision: 0.5757575757575758, Recall: 0.5588235294117647
EM: 0.0, F1: 0.55999999995072, Precision: 0.5, Recall: 0.6363636363636364
EM: 0.0, F1: 0.09999999998199999, Precision: 0.05555555555555555, Recall: 0.5
EM: 0.0, F1: 0.8559999999501567, Precision: 0.8106060606060606, Recall: 0.906779661

研究生学籍管理问答系统不像医疗法律需要严格的精确度，答案相对来说较长，LLM提供的信息尽可能多包含更加正确答案即可，因此使用F1-score作为指标。
